In [ ]:
#include all imported libraries here
Car Luxury car Real Estate Family car Property Suburb House Neighbourhood Cottage M  Siding

In [1]:
import pandas as pd
import numpy as np

dfOverbrook = pd.read_csv('extractedOverbrook.csv')
#df.head()

#enter the address
address_input = input("Please enter the address from the image search bar: ")
#address       = address_input.lower()

#address_to_match = dfOverbrook.loc[dfOverbrook['FULL_ADDRESS_EN']]
#address_match    = address_to_match.lower()

#use 698 Alesther St and 700 Alesther St
     
match = dfOverbrook.loc[dfOverbrook['FULL_ADDRESS_EN']== address_input]    
if not match.empty:
    print('The building information is: ', match.FULL_ADDRESS_EN.to_string(index=False, header=False),', ', match.POSTAL_CODE.to_string(index=False, header=False), ',', match.MUNICIPALITY.to_string(index=False, header=False), ', Ward',  match.WARD.values, 'and Minimum lotsize of ',  match.MINLOSIZELEFT.values)
else:
    print('No matches found.')


Please enter the address from the image search bar:  698 Alesther St


The building information is:  698 Alesther St ,  K1K1H9 , Old Ottawa , Ward [13] and Minimum lotsize of  [1000.]


In [2]:
#declare all variables

isbuilding                     = 0
balcony_absent                 = 0
exterior_finish                = 0
building_age                   = 0
built_before_1990              = 0
building_height_less_than_four = 0

#subject to change
web_entities_front = input("The descriptions are: ")
lowercase_entities = web_entities_front.lower()
lowercase_entities = web_entities_front.lower()

#list of specific words that relate 
specific_words = ["building", "home", "house", "residence", \
                  "dwelling", "domicile", "lodge", "manor", \
                  "villa", "bungalow", "cottage", "homestead", \
                  "farmhouse", "cabin", "apartment"]

#to train the model
import nltk

entities_token = nltk.word_tokenize(lowercase_entities)
#nltk.download('omw-1.4')
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in entities_token]
print(lemmatized_tokens)

#check if the image returned is a building
match_found =False
#loop through each specific word and check if it's in the returned word set
for word1 in lemmatized_tokens:
    for word2 in specific_words:
        if word1 == word2:
            isbuilding = 1
            print("The image returned contains a building")
            match_found =True
            break
            
    if match_found:
        break    
else:
    print("Not a building!")
#if it is a building then do the following 
if isbuilding == 1:
    #check if balcony is absent   
    balcony_input = input("Is there a balcony? enter y for yes and n for no:")
    if balcony_input == "y":
        balcony_absent = 0
    if balcony_input == "n": 
        balcony_absent = 1
    else:
        print("Please enter y or n")
        balcony_input = input("Is there a balcony? enter y for yes and n for no:")    
    
    #checking the exterior finish of the building
    exterior_type       = ["wood","stucco","metal","vinyl","glass","brick","stone", "other"]
    print(exterior_type)
    exterior_type_input = input("Please enter the exterior finish of the building from the dispalyed list:")
    type_input          = exterior_type_input.lower()

    #checking if the exterior type meets the requirement
    if type_input in ("wood","stucco","stone", "vinyl"):
        exterior_finish = 1
    if type_input in ("metal","glass","brick", "other"):
        exterior_finish = 0    
    else:
        print("Please make sure to check your input")  
    #checking the age of the building
    
    from datetime import datetime
    #get current year
    current_year = datetime.now().year

    age_known_input   = input("Enter y for yes and n for no if the building age and or year built is known") 
    age_known         = age_known_input.lower()

    if age_known == "y":
        year_built_input = int(input("Please enter the year built: "))
        building_age = current_year - year_built_input
        if year_built_input <= 1990:
            built_before_1990 = 1
        else:
            built_before_1990 = 0
    else : 
        building_age      = 0
        built_before_1990 = 0
        print('The result might not be as accurate since the Year built is not known.')

    #check if the building is more less than four stories
    absolute_zones  = ('R1','R2','R3')
    column_to_check = dfOverbrook['RESIDENTIAL_ZONING']
    bool_column     = column_to_check == 'R4'
    if column_to_check.isin(absolute_zones).any():
        building_height_less_than_four = 1
    if bool_column.any():
        print("Ur here!")
        building_height_input = input("Is the building less than four (4) stories? y for yes and n for no: ")
        building_height       = building_height_input.lower()
    
        if building_height == "y":
            building_height_less_than_four = 1
        else:
            building_height_less_than_four = 0
    else:
        print("The zoning information does not fit into the preferred category ")
        building_height_less_than_four = 0
else:
    balcony_absent                 = 0
    exterior_finish                = 0
    building_age                   = 0
    built_before_1990              = 0
    building_height_less_than_four = 0
    
#create the input to feed into the model
model_input = [isbuilding,building_age,built_before_1990,building_height_less_than_four,balcony_absent,exterior_finish]
print(model_input)

The descriptions are:  Car Luxury car Real Estate Family car Property Suburb House Neighbourhood Cottage M  Siding


['car', 'luxury', 'car', 'real', 'estate', 'family', 'car', 'property', 'suburb', 'house', 'neighbourhood', 'cottage', 'm', 'siding']
The image returned contains a building


Is there a balcony? enter y for yes and n for no: n


['wood', 'stucco', 'metal', 'vinyl', 'glass', 'brick', 'stone', 'other']


Please enter the exterior finish of the building from the dispalyed list: brick
Enter y for yes and n for no if the building age and or year built is known y
Please enter the year built:  1982


Ur here!


Is the building less than four (4) stories? y for yes and n for no:  y


[1, 41, 1, 1, 1, 0]


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import warnings
#warnings.filterwarnings("ignore")
# import module
from sklearn.model_selection import train_test_split
# import Decision Tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

# next load the data
df = pd.read_csv('ModelFourTrainingDataset.csv')

# drop all variables from the data that we won't need for the model.
df = df.drop(['buildingID', 'latitude', 'longitude', 'postalCode', 'propertyAddress','ImageClassID'], axis = 1)

# seperate input features in x
x = df.drop('suitabilityForRetrofit', axis = 1)

# store the target variable in y
y = df['suitabilityForRetrofit']


# Split the dataset
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.20, random_state=11)

# create an instane of the model
dt = DecisionTreeClassifier(max_depth = 5, max_features=6)

# train the model
dtmodel = dt.fit(xtrain,ytrain)

# Predict on x_test
y_pred = dt.predict(xtest)

# Evaluate the Model
print('Accuracy of Random Forest model on the train set: {:.2f}'.format(dt.score(xtrain, ytrain)))
print('Accuracy of Random Forest model on the test set: {:.2f}'.format(dt.score(xtest, ytest)))
confusion_matrix(ytest, y_pred)



Accuracy of Random Forest model on the train set: 1.00
Accuracy of Random Forest model on the test set: 1.00


array([[12,  0,  0,  0,  0,  0],
       [ 0, 67,  0,  0,  0,  0],
       [ 0,  0, 93,  0,  0,  0],
       [ 0,  0,  0, 48,  0,  0],
       [ 0,  0,  0,  0,  8,  0],
       [ 0,  0,  0,  0,  0, 22]], dtype=int64)

In [4]:
#Save the model
import pickle

#load the model
model4_manual = pickle.load(open('DECISIONTREE_MODEL_MANUAL','rb'))

model4_manual.predict([model_input])

C:\Users\sarah\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['B'], dtype=object)

In [5]:
#Save the model
import pickle

#load the model
model4_manual = pickle.load(open('DECISIONTREE_MODEL_MANUAL','rb'))

result= model4_manual.predict([model_input])

C:\Users\sarah\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [6]:
print(result)

['B']
